In [1]:
!pip install -q kaggle==1.5.12
!pip install -q torch torchvision timm pandas sklearn pillow tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import os

os.environ["KAGGLE_USERNAME"] = "nikhilsanga1206"
os.environ["KAGGLE_KEY"] = "16c0c673e500366baf8a2262a8494565"

print("Kaggle API Ready!")


Kaggle API Ready!


In [3]:
DATASET_SLUG = "khanfashee/nih-chest-x-ray-14-224x224-resized"

!mkdir -p /content/datasets

!kaggle datasets download -d $DATASET_SLUG -p /content/datasets --unzip


100% 2.30G/2.30G [00:20<00:00, 234MB/s]
100% 2.30G/2.30G [00:20<00:00, 121MB/s]


In [4]:
import os

print("Files downloaded:")
for f in os.listdir("/content/datasets"):
    print(f)


Files downloaded:
Data_Entry_2017.csv
images-224
train_val_list_NIH.txt
BBox_List_2017_Official_NIH.csv
test_list_NIH.txt
pretrained_model.h5


In [5]:
import pandas as pd

META_PATH = "/content/datasets/Data_Entry_2017.csv"
meta = pd.read_csv(META_PATH)

# clean column names
meta.columns = meta.columns.str.strip().str.lower().str.replace(" ", "_")

meta.head()


,image_index,finding_labels,follow-up_#,patient_id,patient_age,patient_gender,view_position,originalimage[width,height],originalimagepixelspacing[x,y],unnamed:_11
0,00000001_000.png,Cardiomegaly,0,1,058Y,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,058Y,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,058Y,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,081Y,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,081Y,F,PA,2582,2991,0.143,0.143,NaN


In [6]:
# get all unique disease labels
all_labels = set()

for entry in meta["finding_labels"]:
    labs = entry.split("|")
    for l in labs:
        all_labels.add(l)

all_labels = sorted(list(all_labels))
print("Total classes:", len(all_labels))
print(all_labels)

# one-hot encode all labels
for label in all_labels:
    meta[label] = meta["finding_labels"].apply(lambda x: int(label in x.split("|")))

meta.head()


Total classes: 15
['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


,image_index,finding_labels,follow-up_#,patient_id,patient_age,patient_gender,view_position,originalimage[width,height],originalimagepixelspacing[x,...,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000001_000.png,Cardiomegaly,0,1,058Y,M,PA,2682,2749,0.143,...,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,058Y,M,PA,2894,2729,0.143,...,1,0,0,0,0,0,0,0,0,0
2,00000001_002.png,Cardiomegaly|Effusion,2,1,058Y,M,PA,2500,2048,0.168,...,0,0,0,0,0,0,0,0,0,0
3,00000002_000.png,No Finding,0,2,081Y,M,PA,2500,2048,0.171,...,0,0,0,0,0,1,0,0,0,0
4,00000003_000.png,Hernia,0,3,081Y,F,PA,2582,2991,0.143,...,0,0,1,0,0,0,0,0,0,0


In [7]:
import os

IMG_DIR = "/content/datasets/images-224/images-224"

meta["img_path"] = meta["image_index"].apply(lambda x: os.path.join(IMG_DIR, x))
meta.head()


,image_index,finding_labels,follow-up_#,patient_id,patient_age,patient_gender,view_position,originalimage[width,height],originalimagepixelspacing[x,...,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,img_path
0,00000001_000.png,Cardiomegaly,0,1,058Y,M,PA,2682,2749,0.143,...,0,0,0,0,0,0,0,0,0,/content/datasets/images-224/images-224/000000...
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,058Y,M,PA,2894,2729,0.143,...,0,0,0,0,0,0,0,0,0,/content/datasets/images-224/images-224/000000...
2,00000001_002.png,Cardiomegaly|Effusion,2,1,058Y,M,PA,2500,2048,0.168,...,0,0,0,0,0,0,0,0,0,/content/datasets/images-224/images-224/000000...
3,00000002_000.png,No Finding,0,2,081Y,M,PA,2500,2048,0.171,...,0,0,0,0,1,0,0,0,0,/content/datasets/images-224/images-224/000000...
4,00000003_000.png,Hernia,0,3,081Y,F,PA,2582,2991,0.143,...,0,1,0,0,0,0,0,0,0,/content/datasets/images-224/images-224/000000...


In [8]:
meta = meta[meta["img_path"].apply(os.path.exists)]
len(meta)


112120

In [9]:
train_val_list = open("/content/datasets/train_val_list_NIH.txt").read().splitlines()
test_list = open("/content/datasets/test_list_NIH.txt").read().splitlines()

meta["split"] = meta["image_index"].apply(
    lambda x: "test" if x in test_list else "train_val"
)

train_df = meta[meta["split"] == "train_val"]
test_df  = meta[meta["split"] == "test"]

print(len(train_df), len(test_df))


86524 25596


In [10]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    train_df,
    test_size=0.10,
    random_state=42,
    shuffle=True
)

print("Train:", len(train_df))
print("Val:", len(val_df))


Train: 77871
Val: 8653


In [11]:
label_cols = all_labels   # the 14 classes

print("Label columns:", label_cols)

train_targets = train_df[label_cols].values
val_targets   = val_df[label_cols].values
test_targets  = test_df[label_cols].values


Label columns: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [12]:
from PIL import Image
from torch.utils.data import Dataset

class ChestXrayDataset(Dataset):
    def __init__(self, df, label_cols, transform=None):
        self.df = df.reset_index(drop=True)
        self.label_cols = label_cols
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img = Image.open(row["img_path"]).convert("RGB")
        labels = row[self.label_cols].values.astype("float32")

        if self.transform:
            img = self.transform(img)

        return img, labels


In [14]:
# LABELS = list of all disease columns created earlier
LABELS = all_labels   # from preprocessing step

print("LABELS:", LABELS)
print("Total classes:", len(LABELS))


LABELS: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']
Total classes: 15


In [15]:
from torch.utils.data import DataLoader
import torchvision.transforms as T

# Transforms
train_tf = T.Compose([
    T.Resize((256,256)),
    T.RandomResizedCrop(224, scale=(0.8,1.0)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_tf = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# Create datasets
train_ds = ChestXrayDataset(train_df, LABELS, transform=train_tf)
val_ds   = ChestXrayDataset(val_df, LABELS, transform=val_tf)
test_ds  = ChestXrayDataset(test_df, LABELS, transform=val_tf)

# Create dataloaders
BATCH = 32

train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)
val_dl   = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)
test_dl  = DataLoader(test_ds, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)

print("DataLoaders ready:")
print(" Train batches:", len(train_dl))
print(" Val batches:", len(val_dl))
print(" Test batches:", len(test_dl))


DataLoaders ready:
 Train batches: 2434
 Val batches: 271
 Test batches: 800


In [16]:
import torch
import torch.nn as nn
import torchvision.models as models


In [17]:
NUM_CLASSES = len(LABELS)

def build_resnet50():
    model = models.resnet50(weights="IMAGENET1K_V2")   # pretrained
    in_features = model.fc.in_features

    # replace final layer for multi-label classification
    model.fc = nn.Sequential(
        nn.Linear(in_features, NUM_CLASSES),
        nn.Sigmoid()   # multi-label output
    )

    return model


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model = build_resnet50().to(device)

print("ResNet50 Ready on:", device)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 93.6MB/s]


ResNet50 Ready on: cuda


In [19]:
import torch.nn.functional as F

criterion = nn.BCELoss()   # since our model ends with Sigmoid()
optimizer = torch.optim.AdamW(resnet_model.parameters(), lr=1e-4, weight_decay=1e-4)


In [20]:
def train_one_epoch(model, dl, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for imgs, labels in dl:
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(imgs)  # shape: [batch, 14]
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dl)


def validate(model, dl, criterion, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for imgs, labels in dl:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()

    return total_loss / len(dl)


In [ ]:
EPOCHS = 10
best_val = float("inf")

for epoch in range(1, EPOCHS + 1):
    print(f"\n===== Epoch {epoch}/{EPOCHS} =====")

    train_loss = train_one_epoch(resnet_model, train_dl, optimizer, criterion, device)
    val_loss   = validate(resnet_model, val_dl, criterion, device)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss:   {val_loss:.4f}")

    if val_loss < best_val:
        best_val = val_loss
        torch.save(resnet_model.state_dict(), "/content/resnet50_best.pth")
        print("🔥 Saved New Best ResNet50 Model!")



===== Epoch 1/10 =====
Train Loss: 0.1767
Val Loss:   0.1699
🔥 Saved New Best ResNet50 Model!

===== Epoch 2/10 =====
Train Loss: 0.1634
Val Loss:   0.1691
🔥 Saved New Best ResNet50 Model!

===== Epoch 3/10 =====
Train Loss: 0.1591
Val Loss:   0.1661
🔥 Saved New Best ResNet50 Model!

===== Epoch 4/10 =====
Train Loss: 0.1556
Val Loss:   0.1641
🔥 Saved New Best ResNet50 Model!

===== Epoch 5/10 =====
Train Loss: 0.1527
Val Loss:   0.1645

===== Epoch 6/10 =====
Train Loss: 0.1494
Val Loss:   0.1661

===== Epoch 7/10 =====
Train Loss: 0.1464
Val Loss:   0.1678

===== Epoch 8/10 =====
Train Loss: 0.1431
Val Loss:   0.1748

===== Epoch 9/10 =====
Train Loss: 0.1396
Val Loss:   0.1733

===== Epoch 10/10 =====


In [22]:
import torch

best_resnet = build_resnet50().to(device)
best_resnet.load_state_dict(torch.load("/content/resnet50_best.pth"))
best_resnet.eval()

print("Best ResNet50 loaded!")


Best ResNet50 loaded!


In [23]:
import numpy as np

all_preds = []
all_targets = []

with torch.no_grad():
    for imgs, labels in val_dl:
        imgs = imgs.to(device)
        labels = labels.cpu().numpy()

        outputs = best_resnet(imgs).cpu().numpy()   # sigmoid outputs 0–1

        all_preds.append(outputs)
        all_targets.append(labels)

all_preds = np.vstack(all_preds)
all_targets = np.vstack(all_targets)

print("Shapes:", all_preds.shape, all_targets.shape)


Shapes: (8653, 15) (8653, 15)


In [24]:
binary_preds = (all_preds >= 0.5).astype(int)


In [25]:
from sklearn.metrics import f1_score

f1_micro = f1_score(all_targets, binary_preds, average='micro')
f1_macro = f1_score(all_targets, binary_preds, average='macro')

print("F1 Micro:", f1_micro)
print("F1 Macro:", f1_macro)


F1 Micro: 0.5535653554018819
F1 Macro: 0.17504687913898975


In [26]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(all_targets, all_preds, average='macro')
print("AUC (macro):", auc)


AUC (macro): 0.8302678849832876


Metric	Your Score	Meaning
1.AUC (macro)	0.83
2.F1 Micro	0.55
3.F1 Macro	0.17
4.Accuracy	Not used	Not meaningful for multi-label

✔ AUC = 0.83

This is the best indicator of model quality for this task.
AUC > 0.80 is considered high performance for NIH Chest X-ray dataset.

You have achieved research-level performance.
(For reference, CheXNet paper also reported ~0.83 AUC.)

✔ F1 Micro = 0.55

This means your model is detecting positive labels fairly well overall.

✔ F1 Macro = 0.17

Low because:

Common diseases dominate the dataset

Rare diseases have fewer positives

Macro F1 penalizes that imbalance

This is normal.

⭐ Conclusion:

Your ResNet-50 model is very good and strong enough for deployment.

▶ Next Step: EfficientNet-B0

Now we train EfficientNet and compute:

AUC

F1

Compare with ResNet

The better model becomes your final classification model for deployment.


***Efficient net
***

In [27]:
!pip install -q timm
import timm
import torch.nn as nn


In [28]:
NUM_CLASSES = len(LABELS)

def build_efficientnet_b0():
    model = timm.create_model("efficientnet_b0", pretrained=True)
    in_features = model.classifier.in_features

    model.classifier = nn.Sequential(
        nn.Linear(in_features, NUM_CLASSES),
        nn.Sigmoid()   # multi-label output
    )
    return model


In [29]:
effnet_model = build_efficientnet_b0().to(device)
print("EfficientNet-B0 ready on:", device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

EfficientNet-B0 ready on: cuda


In [30]:
criterion_eff = nn.BCELoss()
optimizer_eff = torch.optim.AdamW(
    effnet_model.parameters(),
    lr=1e-4,
    weight_decay=1e-4
)


In [31]:
EPOCHS = 10
best_val_eff = float("inf")

for epoch in range(1, EPOCHS + 1):
    print(f"\n===== EfficientNet Epoch {epoch}/{EPOCHS} =====")

    train_loss = train_one_epoch(effnet_model, train_dl, optimizer_eff, criterion_eff, device)
    val_loss   = validate(effnet_model, val_dl, criterion_eff, device)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss:   {val_loss:.4f}")

    if val_loss < best_val_eff:
        best_val_eff = val_loss
        torch.save(effnet_model.state_dict(), "/content/efficientnet_b0_best.pth")
        print("🔥 Saved NEW Best EfficientNet Model!")



===== EfficientNet Epoch 1/10 =====


KeyboardInterrupt: 

In [32]:
from tqdm import tqdm

def train_one_epoch(model, dl, optimizer, criterion, device):
    model.train()
    total_loss = 0

    loop = tqdm(dl, total=len(dl), desc="Training", position=0, leave=True)

    for imgs, labels in loop:
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # update progress bar
        loop.set_postfix(loss=loss.item())

    return total_loss / len(dl)


def validate(model, dl, criterion, device):
    model.eval()
    total_loss = 0

    loop = tqdm(dl, total=len(dl), desc="Validating", position=0, leave=True)

    with torch.no_grad():
        for imgs, labels in loop:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            loop.set_postfix(val_loss=loss.item())

    return total_loss / len(dl)


In [ ]:
EPOCHS = 10
best_val_eff = float("inf")

for epoch in range(1, EPOCHS + 1):
    print(f"\n===== EfficientNet Epoch {epoch}/{EPOCHS} =====")

    train_loss = train_one_epoch(effnet_model, train_dl, optimizer_eff, criterion_eff, device)
    val_loss   = validate(effnet_model, val_dl, criterion_eff, device)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss:   {val_loss:.4f}")

    if val_loss < best_val_eff:
        best_val_eff = val_loss
        torch.save(effnet_model.state_dict(), "/content/efficientnet_b0_best.pth")
        print("🔥 Saved NEW Best EfficientNet Model!")



===== EfficientNet Epoch 1/10 =====


Validating: 100%|██████████| 271/271 [00:24<00:00, 11.08it/s, val_loss=0.156]


Train Loss: 0.1788
Val Loss:   0.1729
🔥 Saved NEW Best EfficientNet Model!

===== EfficientNet Epoch 2/10 =====


Validating: 100%|██████████| 271/271 [00:23<00:00, 11.32it/s, val_loss=0.172]


Train Loss: 0.1648
Val Loss:   0.1678
🔥 Saved NEW Best EfficientNet Model!

===== EfficientNet Epoch 3/10 =====


Validating: 100%|██████████| 271/271 [00:24<00:00, 11.21it/s, val_loss=0.169]


Train Loss: 0.1599
Val Loss:   0.1658
🔥 Saved NEW Best EfficientNet Model!

===== EfficientNet Epoch 4/10 =====


Validating: 100%|██████████| 271/271 [00:24<00:00, 11.20it/s, val_loss=0.173]


Train Loss: 0.1560
Val Loss:   0.1656
🔥 Saved NEW Best EfficientNet Model!

===== EfficientNet Epoch 5/10 =====


Training:  86%|████████▋ | 2102/2434 [06:25<01:24,  3.92it/s, loss=0.159]

In [ ]:
model_eff.load_state_dict(torch.load("/content/efficientnet_best.pth"))
model_eff.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for imgs, labels in val_dl:
        imgs = imgs.to(DEVICE)
        outputs = model_eff(imgs).cpu()

        all_preds.append(outputs)
        all_targets.append(labels)

all_preds = torch.cat(all_preds).numpy()
all_targets = torch.cat(all_targets).numpy()


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score

binary_preds = (all_preds > 0.5).astype(int)

print("F1 Micro:", f1_score(all_targets, binary_preds, average="micro"))
print("F1 Macro:", f1_score(all_targets, binary_preds, average="macro"))
print("AUC:", roc_auc_score(all_targets, all_preds, average="macro"))


In [ ]:
from google.colab import files
files.download("/content/efficientnet_best.pth")
